<a href="https://colab.research.google.com/github/ChinmayeeJuturu/DL_ASSIGNMENT2_148/blob/main/DL_ASSIGNMENT_2_QUESTION2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#QUESTION2
# Install necessary libraries

import os
os.environ["WANDB_DISABLED"] = "true"  # ✅ Disable wandb before training

#Imports

import torch
from google.colab import files
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments
)

# Upload your lyrics.txt file
uploaded = files.upload()
lyrics_file = list(uploaded.keys())[0]

# Load and read lyrics file
with open(lyrics_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]  # Remove blank lines

# Convert to Hugging Face dataset
dataset = Dataset.from_dict({"text": lines})

# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Avoid padding error
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data collator (for causal LM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=200,
    logging_steps=20,
    overwrite_output_dir=True,
    save_total_limit=1,
    prediction_loss_only=True,
    logging_dir="./logs"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save model and tokenizer
model.save_pretrained("gpt2-lyrics-finetuned")
tokenizer.save_pretrained("gpt2-lyrics-finetuned")


Saving nursery_rhymes.txt to nursery_rhymes (2).txt


Map:   0%|          | 0/2959 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-4-b7b356eaea9b>:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,4.327700
40,4.148500
60,3.436500
80,3.787100
100,3.931000
120,3.567300
140,3.725400
160,3.616000
180,3.599200
200,3.686500


('gpt2-lyrics-finetuned/tokenizer_config.json',
 'gpt2-lyrics-finetuned/special_tokens_map.json',
 'gpt2-lyrics-finetuned/vocab.json',
 'gpt2-lyrics-finetuned/merges.txt',
 'gpt2-lyrics-finetuned/added_tokens.json')

In [5]:
from transformers import pipeline

# Load fine-tuned model and tokenizer
generator = pipeline("text-generation", model="gpt2-lyrics-finetuned", tokenizer="gpt2-lyrics-finetuned")

# Get user prompt
prompt = input("Enter your song prompt: ")

# Generate lyrics
result = generator(prompt, max_new_tokens=50)

# Print the generated text
print("\n Generated Lyrics:")
print(result[0]['generated_text'])


Device set to use cuda:0


Enter your song prompt: The Queen of Hearts she made some tarts,

 Generated Lyrics:
The Queen of Hearts she made some tarts, some pancakes, some cheese, and they all bubled at once. And all in a bucket; and the Queen of Hearts she said: "Oh yes, I'll do, I'll do." And the worms began multiplying, and the fire soon
